# 🧭 Port-to-Metro Crosswalk (ChatGPT-Enhanced)

This notebook builds a crosswalk between U.S. Census port names and their corresponding **Core-Based Statistical Areas (CBSAs)** or metro areas.

## 🚀 What It Does
- Takes raw port names (e.g., "Los Angeles, CA" or "Port Huron, MI")
- Uses the OpenAI ChatGPT API to infer and assign the most likely metro area (CBSA)
- Outputs a clean, one-to-one mapping for use in trade, logistics, and regional economic analysis

## 🧠 Why It Matters
Census port data is rich, but **not metro-aware**. This crosswalk bridges that gap, enabling local-level aggregation of import/export data by region.

## 🛠️ Requirements
- Jupyter Notebook
- OpenAI API key
- `openai`, `pandas`, `tqdm`

## 📂 Output
- A CSV mapping ports to CBSA names
- Ready for integration into broader tariff or trade models

> ⚠️ Note: This uses a GPT model to make geographic inferences — results are strong but not perfect. Manual review or edits may be needed.


In [28]:
import openai
import pandas as pd
import requests
import json
import os
from io import StringIO


In [98]:


openai_api_key = "" #Add API Key Here
if openai_api_key is None:
    raise ValueError("OpenAI API key is not set in environment variables.")

url = "https://api.openai.com/v1/chat/completions"

def chat_gpt_process(chunk):

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {openai_api_key}"
    }

    data = {
        "model": "gpt-3.5-turbo",
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful assistant from the Census Buereu. You are especially good at understanding CBSA codes. You should respond back in a csv format with sep '|'. Please respond back with format: PORT|PORT_NAME|CBSA_CODE|CBSA_NAME"
            },
            {
                "role": "user",
                "content": f"Do you know the CBSA code and CBSA Name of the following port? {chunk}"
            }
        ]
    }

    response = requests.post(url, headers=headers, json=data)
    #print(response)
    return response

# Check if the request was successful
if response.status_code == 200:
    print("Response from OpenAI:", response.json())
    print('\n')
    print(response.json()['choices'][0]['message']['content'])
else:
    print("Error:", response.status_code, response.text)

Response from OpenAI: {'id': 'chatcmpl-BODzlVYnj1bPnsSmXOaF7QeuLyhTq', 'object': 'chat.completion', 'created': 1745113369, 'model': 'gpt-3.5-turbo-0125', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'CBSA_CODE|CBSA_NAME\n38860|Portland-South Portland, ME', 'refusal': None, 'annotations': []}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 77, 'completion_tokens': 18, 'total_tokens': 95, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}}, 'service_tier': 'default', 'system_fingerprint': None}


CBSA_CODE|CBSA_NAME
38860|Portland-South Portland, ME


In [99]:
df_ports = pd.read_csv('port_names.csv')

In [100]:

all_chunk_results = []  # list to collect DataFrames


# Example function you want to apply
def process_chunk(chunk):
    # do something with the chunk
    print(f"Processing {len(chunk)} rows")
    return chunk  # or return modified chunk

# Assuming your DataFrame is called df_agg
chunk_size = int(len(df_ports) * 0.05)
num_chunks = int(np.ceil(len(df_ports) / chunk_size))

for i in range(num_chunks):
    print(f'running number: {i}')
    start = i * chunk_size
    end = start + chunk_size
    chunk = df_ports.iloc[start:end]
    
    # Apply your function here
    #print(chunk.tolist())
    result = chat_gpt_process(chunk)
    data = result.json()['choices'][0]['message']['content']
    chunk_result = pd.read_csv(StringIO(data),sep='|')
    
    if chunk_result is not None:
        all_chunk_results.append(chunk_result)
    
    
df_cbsa = pd.concat(all_chunk_results, ignore_index=True)    

running number: 0
running number: 1
running number: 2
running number: 3
running number: 4
running number: 5
running number: 6
running number: 7
running number: 8
running number: 9
running number: 10
running number: 11
running number: 12
running number: 13
running number: 14
running number: 15
running number: 16
running number: 17
running number: 18
running number: 19
running number: 20


In [101]:
df_cbsa[pd.notnull(df_cbsa['CBSA_CODE'])].to_csv('temp.csv')

In [103]:
df_cbsa[pd.notnull(df_cbsa['CBSA_CODE'])].to_csv('cbsa_to_port.csv',index=False)

In [87]:
df_cbsa_temp

,PORT,PORT_NAME,CBSA_CODE,CBSA_NAME
0,412,"NEW HAVEN, CT",14860,"New Haven-Milford, CT"
1,413,"NEW LONDON, CT",35300,"New London-Norwich, CT-RI"
2,416,"LAWRENCE, MA",14484,"Boston-Cambridge-Newton, MA-NH"
3,417,"LOGAN AIRPORT, MA",14484,"Boston-Cambridge-Newton, MA-NH"
4,481,"L.G. HANSCOM FIELD, BEDFORD, MA",14454,"Boston-Cambridge-Newton, MA-NH"
5,501,"NEWPORT, RI",39300,"Providence-Warwick, RI-MA"
6,502,"PROVIDENCE, RI",39300,"Providence-Warwick, RI-MA"
7,503,"MELLVILLE, RI",39300,"Providence-Warwick, RI-MA"
8,701,"OGDENSBURG, NY",30700,"Ogdensburg-Massena, NY"
9,704,"MASSENA, NY",31700,"Ogdensburg-Massena, NY"


In [62]:

data = data.json()['choices'][0]['message']['content']
df_cbsa = pd.read_csv(StringIO(data2),sep='|')
df_cbsa
#df = pd.DataFrame(data[1:], columns=data[0])

,PORT,PORT_NAME,CBSA_CODE,CBSA_NAME
0,0,TOTAL FOR ALL PORTS,NaN,NaN
1,1,"PORTLAND, ME",38860.0,"Portland-South Portland, ME"
2,2,"BANGOR, ME",NaN,NaN
3,3,"EASTPORT, ME",NaN,NaN
4,4,"JACKMAN, ME",NaN,NaN


In [80]:
df_cbsa_temp

,PORT,PORT_NAME,CBSA_CODE,CBSA_NAME
0,1303,"BALTIMORE, MD",12580,"Baltimore-Columbia-Towson, MD"
1,1304,"CRISFIELD, MD",99999,Not Applicable
2,1305,"BALTIMORE-WASHINGTON INTERNATIONAL AIRPORT, MD",12580,"Baltimore-Columbia-Towson, MD"
3,1401,"NORFOLK-NEWPORT NEWS, VA",46660,"Virginia Beach-Norfolk-Newport News, VA-NC"
4,1404,"RICHMOND-PETERSBURG, VA",36860,"Richmond, VA"
5,1409,"CHARLESTON, WV",34060,"Charleston, WV"
6,1410,"FRONT ROYAL, VA",47900,"Washington-Arlington-Alexandria, DC-VA-MD-WV"
7,1412,"NEW RIVER VALLEY AIRPORT, VA",22260,"Blacksburg-Christiansburg-Radford, VA"
8,1501,"WILMINGTON, NC",47260,"Wilmington, NC"
9,1502,"WINSTON-SALEM, NC",49180,"Winston-Salem, NC"


In [81]:
data

'PORT|PORT_NAME|CBSA_CODE|CBSA_NAME\n1303|BALTIMORE, MD|12580|Baltimore-Columbia-Towson, MD\n1304|CRISFIELD, MD|99999|Not Applicable\n1305|BALTIMORE-WASHINGTON INTERNATIONAL AIRPORT, MD|12580|Baltimore-Columbia-Towson, MD\n1401|NORFOLK-NEWPORT NEWS, VA|46660|Virginia Beach-Norfolk-Newport News, VA-NC\n1404|RICHMOND-PETERSBURG, VA|36860|Richmond, VA\n1409|CHARLESTON, WV|34060|Charleston, WV\n1410|FRONT ROYAL, VA|47900|Washington-Arlington-Alexandria, DC-VA-MD-WV\n1412|NEW RIVER VALLEY AIRPORT, VA|22260|Blacksburg-Christiansburg-Radford, VA\n1501|WILMINGTON, NC|47260|Wilmington, NC\n1502|WINSTON-SALEM, NC|49180|Winston-Salem, NC\n1503|DURHAM, NC|20500|Durham-Chapel Hill, NC\n1511|BEAUFORT-MOREHEAD CITY, NC|39540|Morehead City, NC\n1512|CHARLOTTE, NC|16740|Charlotte-Concord-Gastonia, NC-SC\n1601|CHARLESTON, SC|16700|Charleston-North Charleston, SC\n1602|GEORGETOWN, SC|22180|Georgetown, SC\n1603|GREENVILLE-SPARTANBURG, SC|24860|Greenville-Anderson-Mauldin, SC\n1604|COLUMBIA, SC|25940|Colum